In [3]:
!pip3 install -r requirements.txt

In [4]:
import os
from utilities.data_utils.Dataset import FacialDataset, get_transform
from utilities.utils import collate_fn
from utilities.train_eval.engine import train_one_epoch, evaluate, get_model_result
import glob

import nvidia_smi # for python 3, you need nvidia-ml-py3 library

import torch
torch.cuda.empty_cache()
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead

In [5]:
PATH = 'Weight.pth'

output_image_folder = 'output'

proj = 'fabric' # 'sign','fabric' (put ur project name here)

if proj == 'sign':
    classes = ['warning', 'prohibitory', 'mandatory']
elif proj == 'fabric':
    classes = ['Hole', 'Line', 'Stain']
else:
    classes = ['bg']

num_classes = len(classes)+1  # n class + background

batch_size = 3

num_epochs = 30

In [6]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

Total memory: 8505131008
Free memory: 8161722368
Used memory: 343408640


In [7]:
dataset_train = FacialDataset('data/train', get_transform(horizontal_flip=True),classes=classes)
dataset_test = FacialDataset('data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_train = torch.utils.data.DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0,
        collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


In [9]:
anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                        aspect_ratios=([1.0, 1.0, 1.0, 1.0], 
                                                     [0.8, 1.0, 1.0, 1.0], 
                                                     [1.0, 0.8, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0]))

In [10]:
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [11]:
if os.path.isfile(PATH):
    model.load_state_dict(torch.load(PATH))

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

## Training

In [13]:
for epoch in range(num_epochs):
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    print(f'gpu: {res.gpu}%, gpu-mem: {res.memory}%')

    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=1)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

print("Training complete!")

gpu: 1%, gpu-mem: 0%
Epoch: [0]  [  0/128]  eta: 0:02:06  lr: 0.000044  loss: 1.5663 (1.5663)  loss_classifier: 0.1024 (0.1024)  loss_box_reg: 0.0639 (0.0639)  loss_objectness: 0.3232 (0.3232)  loss_rpn_box_reg: 1.0768 (1.0768)  time: 0.9885  data: 0.2144  max mem: 3684
Epoch: [0]  [  1/128]  eta: 0:01:56  lr: 0.000084  loss: 1.5663 (2.2150)  loss_classifier: 0.0364 (0.0694)  loss_box_reg: 0.0001 (0.0320)  loss_objectness: 0.3232 (0.3992)  loss_rpn_box_reg: 1.0768 (1.7144)  time: 0.9174  data: 0.1433  max mem: 3684
Epoch: [0]  [  2/128]  eta: 0:01:53  lr: 0.000123  loss: 1.8415 (2.0905)  loss_classifier: 0.1024 (0.0884)  loss_box_reg: 0.0639 (0.0586)  loss_objectness: 0.3626 (0.3870)  loss_rpn_box_reg: 1.2409 (1.5566)  time: 0.9018  data: 0.1356  max mem: 3686
Epoch: [0]  [  3/128]  eta: 0:01:49  lr: 0.000162  loss: 1.6185 (1.9725)  loss_classifier: 0.1024 (0.1001)  loss_box_reg: 0.0597 (0.0588)  loss_objectness: 0.3556 (0.3792)  loss_rpn_box_reg: 1.0768 (1.4344)  time: 0.8796  data: 0

Epoch: [0]  [ 33/128]  eta: 0:01:25  lr: 0.001342  loss: 2.2024 (2.3893)  loss_classifier: 0.0636 (0.0914)  loss_box_reg: 0.0435 (0.0536)  loss_objectness: 0.3977 (0.3980)  loss_rpn_box_reg: 1.7627 (1.8463)  time: 0.8950  data: 0.1262  max mem: 3693
Epoch: [0]  [ 34/128]  eta: 0:01:24  lr: 0.001382  loss: 2.3158 (2.3872)  loss_classifier: 0.0636 (0.0912)  loss_box_reg: 0.0435 (0.0533)  loss_objectness: 0.3977 (0.3966)  loss_rpn_box_reg: 1.8374 (1.8460)  time: 0.8976  data: 0.1287  max mem: 3693
Epoch: [0]  [ 35/128]  eta: 0:01:23  lr: 0.001421  loss: 2.5753 (2.4219)  loss_classifier: 0.0636 (0.0925)  loss_box_reg: 0.0445 (0.0538)  loss_objectness: 0.4020 (0.4000)  loss_rpn_box_reg: 1.9844 (1.8756)  time: 0.9062  data: 0.1368  max mem: 3694
Epoch: [0]  [ 36/128]  eta: 0:01:23  lr: 0.001460  loss: 2.5848 (2.4494)  loss_classifier: 0.0636 (0.0932)  loss_box_reg: 0.0445 (0.0553)  loss_objectness: 0.3977 (0.3979)  loss_rpn_box_reg: 2.0666 (1.9030)  time: 0.9152  data: 0.1460  max mem: 3694


Epoch: [0]  [ 66/128]  eta: 0:00:55  lr: 0.002640  loss: 1.6246 (2.2228)  loss_classifier: 0.0615 (0.0796)  loss_box_reg: 0.0437 (0.0489)  loss_objectness: 0.2585 (0.3584)  loss_rpn_box_reg: 1.2790 (1.7359)  time: 0.8861  data: 0.1183  max mem: 3694
Epoch: [0]  [ 67/128]  eta: 0:00:55  lr: 0.002679  loss: 1.6270 (2.2232)  loss_classifier: 0.0661 (0.0795)  loss_box_reg: 0.0405 (0.0487)  loss_objectness: 0.2640 (0.3590)  loss_rpn_box_reg: 1.3068 (1.7360)  time: 0.8920  data: 0.1234  max mem: 3694
Epoch: [0]  [ 68/128]  eta: 0:00:54  lr: 0.002719  loss: 1.6270 (2.2239)  loss_classifier: 0.0661 (0.0817)  loss_box_reg: 0.0405 (0.0495)  loss_objectness: 0.2640 (0.3648)  loss_rpn_box_reg: 1.2790 (1.7279)  time: 0.8898  data: 0.1207  max mem: 3694
Epoch: [0]  [ 69/128]  eta: 0:00:53  lr: 0.002758  loss: 1.6270 (2.2203)  loss_classifier: 0.0661 (0.0809)  loss_box_reg: 0.0405 (0.0489)  loss_objectness: 0.2640 (0.3713)  loss_rpn_box_reg: 1.1885 (1.7193)  time: 0.8826  data: 0.1138  max mem: 3694


Epoch: [0]  [ 99/128]  eta: 0:00:26  lr: 0.003938  loss: 2.6212 (2.4101)  loss_classifier: 0.0809 (0.0837)  loss_box_reg: 0.0443 (0.0534)  loss_objectness: 0.3863 (0.3933)  loss_rpn_box_reg: 1.9944 (1.8797)  time: 0.9237  data: 0.1511  max mem: 3700
Epoch: [0]  [100/128]  eta: 0:00:25  lr: 0.003977  loss: 2.5566 (2.4084)  loss_classifier: 0.0809 (0.0840)  loss_box_reg: 0.0443 (0.0539)  loss_objectness: 0.3863 (0.3926)  loss_rpn_box_reg: 1.9738 (1.8780)  time: 0.9296  data: 0.1567  max mem: 3700
Epoch: [0]  [101/128]  eta: 0:00:24  lr: 0.004017  loss: 2.6212 (2.4181)  loss_classifier: 0.0909 (0.0842)  loss_box_reg: 0.0461 (0.0538)  loss_objectness: 0.3863 (0.3957)  loss_rpn_box_reg: 1.9738 (1.8844)  time: 0.9273  data: 0.1542  max mem: 3700
Epoch: [0]  [102/128]  eta: 0:00:23  lr: 0.004056  loss: 2.6212 (2.4163)  loss_classifier: 0.0909 (0.0838)  loss_box_reg: 0.0461 (0.0536)  loss_objectness: 0.3861 (0.3956)  loss_rpn_box_reg: 1.9738 (1.8832)  time: 0.9283  data: 0.1546  max mem: 3700


Epoch: [1]  [  0/128]  eta: 0:01:44  lr: 0.005000  loss: 3.2211 (3.2211)  loss_classifier: 0.0253 (0.0253)  loss_box_reg: 0.0081 (0.0081)  loss_objectness: 0.6260 (0.6260)  loss_rpn_box_reg: 2.5618 (2.5618)  time: 0.8138  data: 0.0323  max mem: 3700
Epoch: [1]  [  1/128]  eta: 0:01:58  lr: 0.005000  loss: 2.2902 (2.7556)  loss_classifier: 0.0253 (0.0408)  loss_box_reg: 0.0081 (0.0165)  loss_objectness: 0.6260 (0.7288)  loss_rpn_box_reg: 1.3774 (1.9696)  time: 0.9292  data: 0.1609  max mem: 3700
Epoch: [1]  [  2/128]  eta: 0:02:01  lr: 0.005000  loss: 2.2902 (2.3269)  loss_classifier: 0.0563 (0.0496)  loss_box_reg: 0.0248 (0.0281)  loss_objectness: 0.6260 (0.6568)  loss_rpn_box_reg: 1.3774 (1.5926)  time: 0.9627  data: 0.1875  max mem: 3700
Epoch: [1]  [  3/128]  eta: 0:02:06  lr: 0.005000  loss: 2.2902 (2.3231)  loss_classifier: 0.0563 (0.0569)  loss_box_reg: 0.0248 (0.0293)  loss_objectness: 0.5127 (0.5787)  loss_rpn_box_reg: 1.3774 (1.6581)  time: 1.0091  data: 0.2388  max mem: 3700


...

Epoch: [16]  [  0/128]  eta: 0:01:51  lr: 0.000000  loss: 0.5085 (0.5085)  loss_classifier: 0.0264 (0.0264)  loss_box_reg: 0.0146 (0.0146)  loss_objectness: 0.0754 (0.0754)  loss_rpn_box_reg: 0.3920 (0.3920)  time: 0.8717  data: 0.1731  max mem: 3705
Epoch: [16]  [  1/128]  eta: 0:01:45  lr: 0.000000  loss: 0.5085 (0.9240)  loss_classifier: 0.0264 (0.0457)  loss_box_reg: 0.0146 (0.0259)  loss_objectness: 0.0754 (0.1153)  loss_rpn_box_reg: 0.3920 (0.7372)  time: 0.8307  data: 0.1273  max mem: 3705
Epoch: [16]  [  2/128]  eta: 0:01:44  lr: 0.000000  loss: 0.7426 (0.8636)  loss_classifier: 0.0501 (0.0472)  loss_box_reg: 0.0333 (0.0283)  loss_objectness: 0.1033 (0.1113)  loss_rpn_box_reg: 0.5559 (0.6767)  time: 0.8293  data: 0.1174  max mem: 3705
Epoch: [16]  [  3/128]  eta: 0:01:49  lr: 0.000000  loss: 0.7426 (1.1067)  loss_classifier: 0.0501 (0.0533)  loss_box_reg: 0.0333 (0.0403)  loss_objectness: 0.1033 (0.1269)  loss_rpn_box_reg: 0.5559 (0.8861)  time: 0.8742  data: 0.1596  max mem: 3

Epoch: [16]  [ 33/128]  eta: 0:01:22  lr: 0.000000  loss: 1.1763 (1.3946)  loss_classifier: 0.0470 (0.0541)  loss_box_reg: 0.0373 (0.0461)  loss_objectness: 0.1285 (0.1642)  loss_rpn_box_reg: 0.9670 (1.1302)  time: 0.8714  data: 0.1552  max mem: 3705
Epoch: [16]  [ 34/128]  eta: 0:01:21  lr: 0.000000  loss: 1.3968 (1.3947)  loss_classifier: 0.0530 (0.0548)  loss_box_reg: 0.0461 (0.0471)  loss_objectness: 0.1329 (0.1634)  loss_rpn_box_reg: 1.0896 (1.1294)  time: 0.8656  data: 0.1483  max mem: 3705
Epoch: [16]  [ 35/128]  eta: 0:01:20  lr: 0.000000  loss: 1.1763 (1.3836)  loss_classifier: 0.0530 (0.0540)  loss_box_reg: 0.0461 (0.0464)  loss_objectness: 0.1285 (0.1608)  loss_rpn_box_reg: 0.9670 (1.1224)  time: 0.8681  data: 0.1496  max mem: 3705
Epoch: [16]  [ 36/128]  eta: 0:01:20  lr: 0.000000  loss: 1.3867 (1.3837)  loss_classifier: 0.0530 (0.0538)  loss_box_reg: 0.0553 (0.0469)  loss_objectness: 0.1329 (0.1604)  loss_rpn_box_reg: 1.0896 (1.1225)  time: 0.8800  data: 0.1610  max mem: 3

Epoch: [16]  [ 66/128]  eta: 0:00:54  lr: 0.000000  loss: 1.2129 (1.3620)  loss_classifier: 0.0440 (0.0511)  loss_box_reg: 0.0334 (0.0444)  loss_objectness: 0.1475 (0.1651)  loss_rpn_box_reg: 0.9701 (1.1014)  time: 0.8847  data: 0.1698  max mem: 3705
Epoch: [16]  [ 67/128]  eta: 0:00:53  lr: 0.000000  loss: 1.2129 (1.3691)  loss_classifier: 0.0449 (0.0523)  loss_box_reg: 0.0335 (0.0457)  loss_objectness: 0.1389 (0.1647)  loss_rpn_box_reg: 0.9701 (1.1064)  time: 0.8928  data: 0.1779  max mem: 3705
Epoch: [16]  [ 68/128]  eta: 0:00:52  lr: 0.000000  loss: 1.2129 (1.3664)  loss_classifier: 0.0449 (0.0519)  loss_box_reg: 0.0334 (0.0454)  loss_objectness: 0.1389 (0.1637)  loss_rpn_box_reg: 1.0207 (1.1054)  time: 0.8856  data: 0.1712  max mem: 3705
Epoch: [16]  [ 69/128]  eta: 0:00:51  lr: 0.000000  loss: 1.1829 (1.3538)  loss_classifier: 0.0433 (0.0514)  loss_box_reg: 0.0311 (0.0450)  loss_objectness: 0.1256 (0.1621)  loss_rpn_box_reg: 0.9701 (1.0953)  time: 0.8871  data: 0.1725  max mem: 3

Epoch: [16]  [ 99/128]  eta: 0:00:26  lr: 0.000000  loss: 0.9851 (1.3686)  loss_classifier: 0.0489 (0.0501)  loss_box_reg: 0.0369 (0.0445)  loss_objectness: 0.1067 (0.1558)  loss_rpn_box_reg: 0.7590 (1.1183)  time: 0.9830  data: 0.2075  max mem: 3705
Epoch: [16]  [100/128]  eta: 0:00:25  lr: 0.000000  loss: 0.9851 (1.3688)  loss_classifier: 0.0372 (0.0500)  loss_box_reg: 0.0319 (0.0442)  loss_objectness: 0.1067 (0.1556)  loss_rpn_box_reg: 0.7590 (1.1190)  time: 0.9865  data: 0.2035  max mem: 3705
Epoch: [16]  [101/128]  eta: 0:00:24  lr: 0.000000  loss: 0.9851 (1.3795)  loss_classifier: 0.0372 (0.0498)  loss_box_reg: 0.0369 (0.0442)  loss_objectness: 0.1067 (0.1561)  loss_rpn_box_reg: 0.7590 (1.1293)  time: 0.9980  data: 0.2106  max mem: 3705
Epoch: [16]  [102/128]  eta: 0:00:23  lr: 0.000000  loss: 0.9677 (1.3737)  loss_classifier: 0.0372 (0.0502)  loss_box_reg: 0.0369 (0.0446)  loss_objectness: 0.1070 (0.1556)  loss_rpn_box_reg: 0.7455 (1.1232)  time: 1.0050  data: 0.2097  max mem: 3

KeyboardInterrupt: 

In [14]:
# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)

## Evaluate

In [15]:
evaluate(model, data_loader_test, device=device)

n_threads =  1
creating index...
index created!
Test:  [ 0/24]  eta: 0:00:02  model_time: 0.1088 (0.1088)  evaluator_time: 0.0014 (0.0014)  time: 0.1169  data: 0.0057  max mem: 3705
Test:  [23/24]  eta: 0:00:00  model_time: 0.1093 (0.1094)  evaluator_time: 0.0012 (0.0024)  time: 0.1249  data: 0.0118  max mem: 3705
Test: Total time: 0:00:02 (0.1246 s / it)
Averaged stats: model_time: 0.1093 (0.1094)  evaluator_time: 0.0012 (0.0024)
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.109
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.389
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.040
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.189
 Average Recall  

In [16]:
torch.cuda.synchronize()
# create directory for saving the model
print("Saving model...")
torch.save(model.state_dict(), PATH)
print("Model saving complete!")
nvidia_smi.nvmlShutdown()

Saving model...
Model saving complete!


## Inference 
Repeating all the process to ensure nothing is taken from training stage
Check output folder for results

In [17]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

dataset_test = FacialDataset('data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                   aspect_ratios=([1.0, 1.0, 1.0, 1.0],
                                                  [0.8, 1.0, 1.0, 1.0],
                                                  [1.0, 0.8, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0]))
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load(PATH))
model.to(device)

# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)
# write testing result to output folder
for img_idx, batch_sampler in enumerate(data_loader_test):
    img_test = batch_sampler[0][0]
    target_test = batch_sampler[1][0]
    i = target_test["image_id"].item()
    get_model_result(img_test, model, target_test, i, device, location=output_image_folder, threshold=0.15,classes=classes)

print("Testing complete!")

torch.cuda.synchronize()
nvidia_smi.nvmlShutdown()

Total memory: 8505131008
Free memory: 4452777984
Used memory: 4052353024
initial boxes: ['Hole', 'Hole', 'Hole', 'Hole', 'Hole', 'Stain', 'Hole', 'Stain', 'Hole', 'Stain']
initial boxes: ['Hole', 'Stain', 'Hole', 'Hole', 'Stain', 'Hole', 'Stain', 'Stain', 'Stain', 'Stain', 'Hole', 'Hole']
initial boxes: ['Stain', 'Line', 'Line', 'Stain', 'Stain', 'Line', 'Hole']
initial boxes: ['Stain', 'Stain', 'Line', 'Line', 'Stain']
initial boxes: ['Stain', 'Stain', 'Line', 'Stain', 'Hole']
initial boxes: ['Stain', 'Line', 'Stain']
initial boxes: ['Hole', 'Hole', 'Stain', 'Hole', 'Hole', 'Stain', 'Hole']
initial boxes: ['Stain', 'Stain', 'Line', 'Line', 'Stain']
initial boxes: ['Hole', 'Stain', 'Line', 'Stain', 'Hole', 'Hole', 'Line', 'Stain']
initial boxes: ['Stain', 'Stain', 'Stain', 'Line', 'Hole', 'Line', 'Stain']
initial boxes: ['Stain', 'Line', 'Stain']
initial boxes: ['Stain', 'Line', 'Stain', 'Hole']
initial boxes: ['Stain', 'Line', 'Hole', 'Stain']
initial boxes: ['Stain', 'Stain', 'Line',